<a href="https://colab.research.google.com/github/ZhouyaoXie/interactive-immersive-multimedia-generation/blob/main/image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *FuseDream*: Training-Free Text-to-Image Generation with Improved CLIP+GAN Space Optimization.

By Xingchao Liu, Chengyue Gong, Lemeng Wu, Shujian Zhang, Hao Su and Qiang Liu from UCSD and UT Austin. (https://github.com/gnobitab/FuseDream). 

Following the commands in order to set up the environment and generate images with text queries using *FuseDream*.

This Colab notebook is the single image version of *FuseDream*. *FuseDream-Composition* will be shared in another Colab notebook.

A baseline method (BigSleep) was provided by https://twitter.com/advadnoun.


In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.


In [1]:
!nvidia-smi

Sun May  1 13:11:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/gnobitab/FuseDream.git
!pip install ftfy regex tqdm numpy scipy h5py lpips==0.1.4
!pip install git+https://github.com/openai/CLIP.git
!pip install gdown
!gdown 'https://drive.google.com/uc?id=17ymX6rhsgHDZw_g5XgAFW4xLSDocARCM'
!gdown 'https://drive.google.com/uc?id=1sOZ9og9kJLsqMNhaDnPJgzVsBZQ1sjZ5'

Cloning into 'FuseDream'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 8.24 MiB | 24.67 MiB/s, done.
Resolving deltas: 100% (40/40), done.
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hczawsdh
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-hczawsdh
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369389 sha256=fca14d0683407f98124548c9067fa209d39fbb8e2a06a9e97318cdeb65c2e589
  Stored in directory: /tmp/pip-ephem-wheel-cache-r4vqfq1q/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
Downloading...
From: https://drive.google.com/uc?id=17ymX6rhsgHDZw_g5XgAFW4xLSDocARCM
To: /content/biggan-256.pth
100% 329M/329M [00:01<00:00, 195MB/s]
Downloading...
From: https://drive

In [3]:
!ls
!cp biggan-256.pth FuseDream/BigGAN_utils/weights/
!cp biggan-512.pth FuseDream/BigGAN_utils/weights/
%cd FuseDream

biggan-256.pth	biggan-512.pth	FuseDream  sample_data
/content/FuseDream


In [4]:
import torch
from tqdm import tqdm
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torchvision
import BigGAN_utils.utils as utils
import clip
import torch.nn.functional as F
from DiffAugment_pytorch import DiffAugment
import numpy as np
from fusedream_utils import FuseDreamBaseGenerator, get_G, save_image

### Setting up parameters
1. SENTENCE: The query text for generating the image. Note: we find that putting a period '.' at the end of the sentence can boost the quality of the generated images, e.g., 'A photo of a blue dog.' generates better images than 'A photo of a blue dog'.
2. INIT_ITERS: Controls the number of images used for initialization (M in the paper, and M = INIT_ITERS*10). Use the default number 1000 should work well.
3. OPT_ITERS: Controls the number of iterations for optimizing the latent variables. Use the default number 1000 should work well.
4. NUM_BASIS: Controls the number of basis images used in optimization (k in the paper). Choose from 5, 10, 15 should work well.
5. MODEL: Currently please choose from 'biggan-256' and 'biggan-512'.
6. SEED: Random seed. Choose an arbitrary integer you like.

In [5]:
#@title Parameters
SENTENCE = "Ill be rolling next summer" #@param {type:"string"}
INIT_ITERS =  200#@param {type:"number"}
OPT_ITERS = 500#@param {type:"number"}
NUM_BASIS = 5#@param {type:"number"}
MODEL = "biggan-512" #@param ["biggan-256","biggan-512"]
SEED = 42#@param {type:"number"}

import sys
sys.argv = [''] ### workaround to deal with the argparse in Jupyter

In [1]:
### Generation: Click the 'run' button and the final generated image will be shown after the end of the algorithm
utils.seed_rng(SEED) 

sentences = ["And Im sitting on the sofa", "Wonderin why Im feelin so good", "On my own", "Come around you, little girl",
             "Bring me my broom", "Leave me by your door", "I’ll do anything for you", "You may call me Terry, Terry, hey, hey, hey",
             "Whatever you need, you can always come with me", "When the deal goes down", "Well, I will pawn my watch", 
             "And chain it to my waist", "You’ll be rolling by me pretty fast", "You’ll catch my bus", "When I’m in L.A",
             "Oh, you will ride by me"]

if MODEL == "biggan-256":
    G, config = get_G(256) 
elif MODEL == "biggan-512":
    G, config = get_G(512) 
else:
    raise Exception('Model not supported')
generator = FuseDreamBaseGenerator(G, config, 10)

import os
from IPython import display

if not os.path.exists('./samples'):
    os.mkdir('./samples')

for sentence in sentences:

    print('Generating:', sentence)
    z_cllt, y_cllt = generator.generate_basis(sentence, init_iters=INIT_ITERS, num_basis=NUM_BASIS)

    z_cllt_save = torch.cat(z_cllt).cpu().numpy()
    y_cllt_save = torch.cat(y_cllt).cpu().numpy()
    img, z, y = generator.optimize_clip_score(z_cllt, y_cllt, sentence, latent_noise=False, augment=True, opt_iters=OPT_ITERS, optimize_y=True)
    ### Set latent_noise = True yields slightly higher AugCLIP score, but slightly lower image quality. We set it to False for dogs.
    score = generator.measureAugCLIP(z, y, sentence, augment=True, num_samples=20)
    print('AugCLIP score:', score)
    save_image(img, 'samples/fusedream_%s_seed_%d_score_%.4f.png'%(sentence, SEED, score))

    display.display(display.Image('samples/fusedream_%s_seed_%d_score_%.4f.png'%(sentence, SEED, score)))

NameError: ignored